In [2]:
# Web scraping (Kyle, due 02/26/2021)
#     BeautifulSoup for web scraping

from urllib.request import urlopen
from bs4 import BeautifulSoup
# from google.colab import files
import pandas as pd
import io
import numpy as np
import dateutil
from dateutil import parser
import textblob
from textblob import TextBlob
import nltk
import requests
from textblob import Word
import matplotlib.pyplot as plt
from nltk.stem.snowball import SnowballStemmer
nltk.download('punkt')

# upload file
#uploaded = files.upload()
#read array
#df2 = pd.read_csv(io.BytesIO(uploaded['logs - Sheet1.csv']))
#turn to flat list
#array_pages = df2.to_numpy()
#array_pages = np.ndarray.tolist(array_pages)
#array_pages = [item for sublist in array_pages for item in sublist]

#url core needed to pull
website = "https://gcsweb-ci.apps.ci.l2s4.p1.openshiftapps.com"
base = "https://gcsweb-ci.apps.ci.l2s4.p1.openshiftapps.com/gcs/origin-ci-test/logs/canary-release-openshift-origin-installer-e2e-aws-4.5-cnv/"
ending = "build-log.txt"
url = "https://gcsweb-ci.apps.ci.l2s4.p1.openshiftapps.com/gcs/origin-ci-test/logs/canary-release-openshift-origin-installer-e2e-aws-4.5-cnv/1300557127638585344/build-log.txt"
page = requests.get(base)    
data = page.text
soup = BeautifulSoup(data)
links = []
for link in soup.find_all('a'):
    links.append(link.get('href'))
links = links[1:-1]

final_array = []
# create array of urls
for x in range(len(links)):
  final_array.append(str(website) + str(links[x]) + str(ending))


# pull all urls logs and store in 2-d array where array_of_logs[x] is a build-log file and 
# array_of_logs[x][y] is an individual log line split by new line
array_of_logs = []
for x in range(len(final_array)):
  page = urlopen(final_array[x])
  html_bytes = page.read()
  array_of_logs.append(str(html_bytes).split('\\n'))
  
# first log
print(array_of_logs[0])


# Analysis on the log data. Trying to find a framework. API (Ningxiao, Parker, Tianze, Hong)
#     Identify limitations with data and potential risks of achieving project goals.


# ******* Tianze *******
#ignore useless "Waiting for setup to finish..."
def ignoreWaiting(logs):
  for i in range(len(logs)):
    if "Waiting for setup to finish..." in logs[i]:
      logs[i]=""
  return logs
# ******* Tianze *******


ModuleNotFoundError: No module named 'pandas'

In [3]:
#url core needed to pull
website2 = "https://gcsweb-ci.apps.ci.l2s4.p1.openshiftapps.com"
base2 = "https://gcsweb-ci.apps.ci.l2s4.p1.openshiftapps.com/gcs/origin-ci-test/logs/release-openshift-ocp-installer-e2e-aws-serial-4.1/"
ending2 = "build-log.txt"
page2 = requests.get(base2)    
data2 = page2.text
soup2 = BeautifulSoup(data2)
links2 = []
for link2 in soup2.find_all('a'):
    links2.append(link2.get('href'))
links2 = links2[1:-1]

final_array2 = []
# create array of urls
for x in range(len(links2)):
  final_array2.append(str(website2) + str(links2[x]) + str(ending2))


# pull all urls logs and store in 2-d array where array_of_logs[x] is a build-log file and 
# array_of_logs[x][y] is an individual log line split by new line
array_of_logs2 = []
for x in range(len(final_array2)):
  page2 = urlopen(final_array2[x])
  html_bytes2 = page2.read()
  array_of_logs2.append(str(html_bytes2).split('\\n'))
  
# first log
print(array_of_logs2[0])


# Analysis on the log data. Trying to find a framework. API (Ningxiao, Parker, Tianze, Hong)
#     Identify limitations with data and potential risks of achieving project goals.


# ******* Tianze *******
#ignore useless "Waiting for setup to finish..."
def ignoreWaiting(logs):
  for i in range(len(logs)):
    if "Waiting for setup to finish..." in logs[i]:
      logs[i]=""
  return logs
# ******* Tianze *******


NameError: name 'requests' is not defined

In [ ]:
# ******* PARKER *******
#  helper function detecting if a string is a date / timestamp
def is_date(str):
  try:
    dateutil.parser.parse(str)
    return True
  except:
    return False

# logs variable contains all parsed logs
logs = []
for i in range(len(array_of_logs)):
  #  removing newline characters
  array_of_logs[i] = array_of_logs[i].replace('\\n', ' ')
  array_of_logs[i] = array_of_logs[i].splitlines()

# removes leading 'b from log
  array_of_logs[i] = array_of_logs[i][0][2:]
# splitting each section as its own index (for parsing)
  array_of_logs[i] = array_of_logs[i].split(' ')

#  tmp is log without timestamps
  tmp = []
  for j in range(len(array_of_logs[i])):
    if is_date(array_of_logs[i][j]) == False:
      tmp.append(array_of_logs[i][j])
    else:
      continue
  logs.append(tmp)

# removes whitespace characters and keeps root words
stemmer = SnowballStemmer("english")
for log in logs:
  log[:] = [stemmer.stem(x) for x in log if x != '']

This cell removes dates and timestamps, whitespace and keeps the stems of words

In [ ]:
# ******* PARKER *******
failures = []
successes = []
# word order for matrix: 0: fail, 1: error, 2: success, 3: run, 4: crashloopbackoff
word_matrix = []
for log in logs:
  tmp = ' '.join(log)
  tmp = TextBlob(tmp)
  fail = tmp.word_counts['fail']
  error = tmp.word_counts['error']
  success = tmp.word_counts['success']
  run = tmp.word_counts['run']
  crash = tmp.word_counts['crashloopbackoff']
  word_matrix.append([fail, error, success, run, crash])

# remove noise
word_matrix = word_matrix[2:]

This cell creates a matrix of keywords. each row is a single log entry and each column is a specific keyword

In [ ]:
# ******* PARKER *******
import math
# Latent Semantic Analysis approach

num_of_docs = [0 for i in range(len(word_matrix[0]))]
for i in range(len(word_matrix)):
  for j in range(len(word_matrix[i])):
    if word_matrix[i][j] != 0:
      num_of_docs[j] += 1
for i in range(len(word_matrix)):
  for j in range(len(word_matrix[i])):
    tf = word_matrix[i][j]
    idf = math.log(len(word_matrix) / num_of_docs[j])
    word_matrix[i][j] = tf * idf

# TODO: try to implement anomaly detection with word matrix

In [ ]:
# ******* NINGXIAO *******
# find events in log, display frequecy of each event in a bar chart
import spacy
s = ""
for log in logs:
    temp = ""
    temp = temp.join(log)
    s += temp
# Error Message: Text of length 5496377 exceeds maximum of 1000000.
s = s[0:100000]
nlp = spacy.load("en_core_web_sm")
doc = nlp(s)
dictionary = {};
for token in doc:
    if token.pos_ == 'VERB':
        if token.text not in dictionary:
            dictionary[token.text] = 1
        else:
            dictionary[token.text] += 1
    #print(token.text, toekn.pos_)

#print(dictionary)

keys = list(dictionary.keys())
values = list(dictionary.values())

fig, ax = plt.subplots(figsize=(9.2, 7))  
ax.barh(keys,values)
plt.title('Verbs in logs')
plt.ylabel('Verbs')
plt.xlabel('Frequency')

plt.show()

The image above shows the verbs and their frequency appear in the tokenized log text. It indicates what event and how often it happens in the log.